1. 데이터 RDD 생성 map()
2. collect()
3. blue mnms만 꺼내기
4. 전체 합계 구하기
5. 도시별 합계 구하기

In [1]:
from pyspark import SparkConf, SparkContext
# 스파크 환경 설정 객체 생성
conf = SparkConf().setMaster('local').setAppName('mnm-rdd-counts')
spark = SparkContext(conf=conf).getOrCreate()

24/12/03 16:09:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/03 16:09:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/03 16:09:47 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
import os
directory = os.path.join(os.getcwd(), 'data')
filename = 'mnm_dataset.csv'
filepath = os.path.join(directory, filename)

In [3]:
# RDD 생성
lines = spark.textFile('file:///'+filepath.replace('\\', '/'))
lines.take(5)

['State,Color,Count', 'TX,Red,20', 'NV,Blue,66', 'CO,Blue,79', 'OR,Blue,71']

In [5]:
header = lines.first()
filtered_lines = lines.filter(lambda row : row != header)
filtered_lines.take(5)

['TX,Red,20', 'NV,Blue,66', 'CO,Blue,79', 'OR,Blue,71', 'WA,Yellow,93']

In [27]:
def parse(row):
    fields = row.split(',')
    state = fields[0]
    color = fields[1]
    count = int(fields[2])
    return state, color, count

In [28]:
# RDD 변환
mnm_rdd = filtered_lines.map(parse)
mnm_rdd.collect()

[('TX', 'Red', 20),
 ('NV', 'Blue', 66),
 ('CO', 'Blue', 79),
 ('OR', 'Blue', 71),
 ('WA', 'Yellow', 93),
 ('WY', 'Blue', 16),
 ('CA', 'Yellow', 53),
 ('WA', 'Green', 60),
 ('OR', 'Green', 71),
 ('TX', 'Green', 68),
 ('NV', 'Green', 59),
 ('AZ', 'Brown', 95),
 ('WA', 'Yellow', 20),
 ('AZ', 'Blue', 75),
 ('OR', 'Brown', 72),
 ('NV', 'Red', 98),
 ('WY', 'Orange', 45),
 ('CO', 'Blue', 52),
 ('TX', 'Brown', 94),
 ('CO', 'Red', 82),
 ('CO', 'Red', 12),
 ('CO', 'Red', 17),
 ('OR', 'Green', 16),
 ('AZ', 'Green', 46),
 ('NV', 'Red', 43),
 ('NM', 'Yellow', 15),
 ('WA', 'Red', 12),
 ('OR', 'Green', 13),
 ('CO', 'Blue', 95),
 ('WY', 'Red', 63),
 ('TX', 'Orange', 63),
 ('WY', 'Yellow', 48),
 ('OR', 'Green', 95),
 ('WA', 'Red', 75),
 ('CO', 'Orange', 93),
 ('NV', 'Orange', 10),
 ('WY', 'Green', 15),
 ('WA', 'Green', 99),
 ('CO', 'Blue', 98),
 ('CA', 'Green', 86),
 ('UT', 'Red', 92),
 ('AZ', 'Brown', 16),
 ('CA', 'Red', 100),
 ('UT', 'Red', 77),
 ('TX', 'Yellow', 29),
 ('WA', 'Orange', 73),
 ('WY', 

In [29]:
# blue mns만 필터링
blue_mnm_rdd = mnm_rdd.filter(lambda x: x[1].lower() == 'blue')
blue_mnm_rdd.collect()

[('NV', 'Blue', 66),
 ('CO', 'Blue', 79),
 ('OR', 'Blue', 71),
 ('WY', 'Blue', 16),
 ('AZ', 'Blue', 75),
 ('CO', 'Blue', 52),
 ('CO', 'Blue', 95),
 ('CO', 'Blue', 98),
 ('CA', 'Blue', 13),
 ('NV', 'Blue', 50),
 ('TX', 'Blue', 49),
 ('CA', 'Blue', 34),
 ('UT', 'Blue', 97),
 ('AZ', 'Blue', 59),
 ('OR', 'Blue', 29),
 ('OR', 'Blue', 54),
 ('CO', 'Blue', 52),
 ('WA', 'Blue', 59),
 ('WY', 'Blue', 59),
 ('WY', 'Blue', 53),
 ('WA', 'Blue', 42),
 ('WA', 'Blue', 37),
 ('NV', 'Blue', 49),
 ('NM', 'Blue', 15),
 ('CA', 'Blue', 39),
 ('UT', 'Blue', 48),
 ('AZ', 'Blue', 82),
 ('CO', 'Blue', 14),
 ('WA', 'Blue', 18),
 ('CO', 'Blue', 89),
 ('TX', 'Blue', 80),
 ('WA', 'Blue', 84),
 ('NM', 'Blue', 46),
 ('AZ', 'Blue', 16),
 ('TX', 'Blue', 68),
 ('CA', 'Blue', 99),
 ('WY', 'Blue', 80),
 ('CO', 'Blue', 44),
 ('CO', 'Blue', 90),
 ('TX', 'Blue', 19),
 ('CA', 'Blue', 65),
 ('TX', 'Blue', 98),
 ('NM', 'Blue', 98),
 ('AZ', 'Blue', 24),
 ('NM', 'Blue', 70),
 ('OR', 'Blue', 96),
 ('OR', 'Blue', 37),
 ('WY', 'Blue

In [30]:
# 전체 합계 계산
total_blue_count = blue_mnm_rdd.map(lambda x: x[2]).sum()
total_blue_count

902516

In [35]:
# 색상별 합계 계산
# state, total, 개수 누적
state_review_count = mnm_rdd.map(lambda x: (x[0], (x[2], 1)))

state_reduce_rdd = state_review_count.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]) )
state_reduce_rdd.collect()

[('TX', (556493, 10072)),
 ('NV', (548477, 10025)),
 ('CO', (556302, 10051)),
 ('OR', (542169, 9903)),
 ('WA', (557227, 10065)),
 ('WY', (538975, 9782)),
 ('CA', (561184, 10164)),
 ('AZ', (546812, 10001)),
 ('NM', (553454, 10050)),
 ('UT', (538942, 9886))]

In [ ]:
# 도시별 합계 계산
